# Halogen Bond Analysis with HBAT

This notebook demonstrates halogen bond analysis using HBAT with the PDB structure 4X21.

**4X21**: Crystal structure containing halogen bonding interactions, perfect for demonstrating HBAT's halogen bond detection capabilities.

## What are Halogen Bonds?

Halogen bonds are non-covalent interactions between a halogen atom (X = Cl, Br, I) and an electronegative atom (typically O, N, S). They play important roles in:
- Drug design and medicinal chemistry
- Protein-ligand interactions
- Crystal engineering
- Molecular recognition

The typical geometry is: **C-X···A** where:
- C-X is the halogen donor
- A is the acceptor atom
- The C-X···A angle is typically close to 180°

## Installation

If you haven't installed the required packages, uncomment and run:

In [ ]:
# !pip install hbat py3Dmol pandas jupyter

## Import Libraries

In [ ]:
import hbat
from hbat.core.analysis import NPMolecularInteractionAnalyzer, AnalysisParameters
from hbat.visualization import visualize_structure_with_interactions, visualize_residue_halogen_bonds
import py3Dmol
import pandas as pd
from pathlib import Path

print(f"HBAT version: {hbat.__version__}")

## Load and Analyze PDB Structure

In [ ]:
# Path to PDB file
pdb_file = "../example_pdb_files/4x21.pdb"

# Create analyzer with default parameters
parameters = AnalysisParameters()
analyzer = NPMolecularInteractionAnalyzer(parameters)

# Run analysis
print("Analyzing structure...")
success = analyzer.analyze_file(pdb_file)

if success:
    print("✓ Analysis completed successfully")
else:
    print("✗ Analysis failed")

## Summary Statistics

In [ ]:
summary = analyzer.get_summary()

print("=" * 60)
print(f"Structure: {pdb_file}")
print("=" * 60)
print(f"Hydrogen Bonds:        {summary['hydrogen_bonds']['count']:4d}")
print(f"Halogen Bonds:         {summary['halogen_bonds']['count']:4d}  ⭐")
print(f"π Interactions:        {summary['pi_interactions']['count']:4d}")
print(f"π-π Stacking:          {summary.get('pi_pi_interactions', {}).get('count', 0):4d}")
print(f"Carbonyl n→π*:         {summary.get('carbonyl_interactions', {}).get('count', 0):4d}")
print(f"n→π* Interactions:     {summary.get('n_pi_interactions', {}).get('count', 0):4d}")
print(f"Cooperativity Chains:  {summary['cooperativity_chains']['count']:4d}")
print("=" * 60)
print(f"\n✓ Found {summary['halogen_bonds']['count']} halogen bonds!")

## Halogen Bond Details

Let's examine the halogen bonds in detail.

In [ ]:
# Extract halogen bond information
halogen_bonds = analyzer.halogen_bonds

print(f"Total Halogen Bonds: {len(halogen_bonds)}\n")

if len(halogen_bonds) > 0:
    # Create DataFrame for better visualization
    xb_data = []
    for xb in halogen_bonds:
        xb_data.append({
            'Halogen Donor': f"{xb.halogen.res_name}{xb.halogen.res_seq}{xb.halogen.name}",
            'Halogen Type': xb.halogen.name[0] if xb.halogen.name else 'X',
            'Acceptor': f"{xb.acceptor.res_name}{xb.acceptor.res_seq}:{xb.acceptor.name}",
            'Distance (Å)': round(xb.distance, 2),
            'Angle (°)': round(xb.angle, 1),
            'Bond Type': xb.bond_type
        })
    
    df_halogen = pd.DataFrame(xb_data)
    print("Halogen Bonds:")
    display(df_halogen)
    
    # Statistics by halogen type
    print("\nHalogen Bond Statistics by Halogen Type:")
    print("-" * 40)
    halogen_types = df_halogen['Halogen Type'].value_counts()
    for halogen_type, count in halogen_types.items():
        print(f"{halogen_type:5s}: {count:3d} bonds")
    
    # Distance and angle statistics
    print("\nGeometric Statistics:")
    print("-" * 40)
    print(f"Distance range: {df_halogen['Distance (Å)'].min():.2f} - {df_halogen['Distance (Å)'].max():.2f} Å")
    print(f"Mean distance:  {df_halogen['Distance (Å)'].mean():.2f} Å")
    print(f"Angle range:    {df_halogen['Angle (°)'].min():.1f} - {df_halogen['Angle (°)'].max():.1f}°")
    print(f"Mean angle:     {df_halogen['Angle (°)'].mean():.1f}°")
else:
    print("No halogen bonds found in this structure.")

## 3D Visualization: Halogen Bonds

Visualize the structure with halogen bonds highlighted in orange.

In [ ]:
# Use HBAT's visualization function
viewer = visualize_structure_with_interactions(
    pdb_file,
    analyzer,
    width=800,
    height=600,
    max_hbonds=10,  # Show fewer H-bonds to focus on halogen bonds
    max_pi_interactions=5,
    show_all_halogen_bonds=True,  # Show ALL halogen bonds
    show_water=False
)

viewer.show()

## Focused Halogen Bond Visualization

Create a cleaner visualization showing only halogen bonds and the involved residues.

In [ ]:
# Use HBAT's visualization function for halogen bonds
if len(halogen_bonds) > 0:
    # Get the first halogen bond to extract residue information
    first_xb = halogen_bonds[0]
    
    # Extract chain, residue number, and residue name from the halogen atom
    chain = "A"
    residue_number = "501"
    residue_name = "3WH"
    
    print(f"Halogen-containing residue: Chain {chain}, Residue {residue_number} ({residue_name})")
    print(f"Visualizing halogen bonds from {chain}{residue_number}{residue_name}\\n")
    
    # Visualize halogen bonds involving this residue
    viewer2 = visualize_residue_halogen_bonds(
        pdb_file,
        analyzer,
        chain=chain,
        residue_number=residue_number,
        residue_name=residue_name,
        width=800,
        height=600,
        show_labels=True,
        show_water=False
    )
    
    viewer2.show()
    
    print(f"\\n📊 Visualization Summary:")
    print(f"   Purple residues: Halogen donors")
    print(f"   Cyan residues: Acceptors")
    print(f"   Orange dashed lines: Halogen bonds")
    print(f"   Focus: {chain}{residue_number}{residue_name}")
else:
    print("No halogen bonds to visualize.")

## Residue Pair Analysis

Analyze which residue pairs form halogen bonds.

In [ ]:
if len(halogen_bonds) > 0:
    # Analyze residue pairs
    pair_data = []
    for xb in halogen_bonds:
        donor_res = f"{xb.halogen.res_name}{xb.halogen.res_seq}"
        acceptor_res = f"{xb.acceptor.res_name}{xb.acceptor.res_seq}"
        pair = f"{donor_res} ⟶ {acceptor_res}"
        pair_data.append(pair)
    
    # Count occurrences
    from collections import Counter
    pair_counts = Counter(pair_data)
    
    print("Halogen Bonding Residue Pairs:")
    print("=" * 60)
    for pair, count in pair_counts.most_common():
        print(f"{pair:40s} : {count} bond(s)")
    
    # Residue type analysis
    print("\nDonor Residue Types:")
    donor_types = [xb.halogen.res_name for xb in halogen_bonds]
    donor_counts = Counter(donor_types)
    for res_type, count in donor_counts.most_common():
        print(f"  {res_type:5s}: {count} bonds")
    
    print("\nAcceptor Residue Types:")
    acceptor_types = [xb.acceptor.res_name for xb in halogen_bonds]
    acceptor_counts = Counter(acceptor_types)
    for res_type, count in acceptor_counts.most_common():
        print(f"  {res_type:5s}: {count} bonds")

## Comparison with Hydrogen Bonds

Compare halogen bonds with hydrogen bonds in the structure.

In [ ]:
print("Interaction Type Comparison:")
print("=" * 60)
print(f"Hydrogen Bonds:    {len(analyzer.hydrogen_bonds):4d}")
print(f"Halogen Bonds:     {len(analyzer.halogen_bonds):4d}")
print(f"π Interactions:    {len(analyzer.pi_interactions):4d}")
print("=" * 60)

if len(analyzer.hydrogen_bonds) > 0 and len(analyzer.halogen_bonds) > 0:
    ratio = len(analyzer.hydrogen_bonds) / len(analyzer.halogen_bonds)
    print(f"\nH-bond to X-bond ratio: {ratio:.1f}:1")

# Show sample hydrogen bonds for comparison
if len(analyzer.hydrogen_bonds) > 0:
    print("\nSample Hydrogen Bonds (first 5):")
    hb_data = []
    for hb in analyzer.hydrogen_bonds[:5]:
        hb_data.append({
            'Donor': f"{hb.donor.res_name}{hb.donor.res_seq}:{hb.donor.name}",
            'Acceptor': f"{hb.acceptor.res_name}{hb.acceptor.res_seq}:{hb.acceptor.name}",
            'Distance (Å)': round(hb.distance, 2),
            'Angle (°)': round(hb.angle, 1),
            'Type': hb.bond_type
        })
    df_hbonds = pd.DataFrame(hb_data)
    display(df_hbonds)

## Parameter Sensitivity Analysis

Test how halogen bond detection changes with different parameters.

In [ ]:
# Test different parameter sets
parameter_sets = [
    {'name': 'Default', 'xb_distance': 3.9, 'xb_angle': 150},
    {'name': 'Strict', 'xb_distance': 3.5, 'xb_angle': 160},
    {'name': 'Relaxed', 'xb_distance': 4.2, 'xb_angle': 140},
]

results = []

for param_set in parameter_sets:
    params = AnalysisParameters(
        xb_distance_cutoff=param_set['xb_distance'],
        xb_angle_cutoff=param_set['xb_angle'],
        analysis_mode='complete'
    )
    
    test_analyzer = NPMolecularInteractionAnalyzer(params)
    test_analyzer.analyze_file(pdb_file)
    
    results.append({
        'Parameter Set': param_set['name'],
        'Distance Cutoff (Å)': param_set['xb_distance'],
        'Angle Cutoff (°)': param_set['xb_angle'],
        'Halogen Bonds Found': len(test_analyzer.halogen_bonds)
    })

print("Parameter Sensitivity Analysis:")
print("=" * 70)
df_params = pd.DataFrame(results)
display(df_params)

print("\n💡 Insight: Adjusting cutoff parameters affects the number of detected")
print("   halogen bonds. Stricter criteria find fewer, higher-quality bonds.")

## Export Results

Save the analysis results for further processing.

In [ ]:
from hbat.export.results import export_to_json_single_file

output_file = "4x21_halogen_bonds_analysis.json"
export_to_json_single_file(analyzer, output_file, pdb_file)
print(f"✓ Results exported to {output_file}")

# Also save halogen bonds to CSV
if len(halogen_bonds) > 0:
    csv_file = "4x21_halogen_bonds.csv"
    df_halogen.to_csv(csv_file, index=False)
    print(f"✓ Halogen bonds exported to {csv_file}")

## Summary

This notebook demonstrated:

1. ✓ Loading and analyzing a PDB structure with halogen bonds (4X21)
2. ✓ Detecting and characterizing halogen bonds
3. ✓ Statistical analysis of halogen bond geometry (distance, angle)
4. ✓ 3D visualization of halogen bonds using py3Dmol
5. ✓ Residue pair analysis for halogen bonding
6. ✓ Comparison with hydrogen bonds
7. ✓ Parameter sensitivity analysis
8. ✓ Exporting results to JSON and CSV formats

### Key Findings:

- Halogen bonds are visualized as **orange dashed lines**
- HBAT supports detection of Cl, Br, and I halogen bonds
- Default parameters: distance ≤ 3.9 Å, angle ≥ 150°
- Halogen bonds complement hydrogen bonds in protein stability

For more information, visit the [HBAT documentation](https://hbat.abhishek-tiwari.com/).